In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from test_utils import test
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [211]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))   
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [212]:
def hinge_loss(y,X,w):
    #z=(y.dot(X)).dot(w.T)
    z=y*(X@w) 
    #print(z)
    hinge=np.maximum(0,1-z) #pour chaque label, max entre 0 et 1-z 
    return hinge

In [137]:
y_test = np.array([1, -1])
x_test = np.array([[1, 2, 3], [4, 5, 6]])
w_test = np.array([0, 1, 5])

In [138]:
print(y_test.shape, x_test.shape, w_test.shape)

(2,) (2, 3) (3,)


In [139]:
print(hinge_loss(y_test, x_test, w_test))
print(hinge_loss(y_test, x_test, w_test).shape)

[ 17 -35]
[ 0 36]
[ 17 -35]
(2,)


In [104]:
print((np.linalg.norm(w_test,2))**2)
np.sum(w_test**2)

25.999999999999996


26

In [213]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective, equation (1) in the exercise pdf),
        that is loss plus regularizer.
        
    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
    
    Returns:
        scalar, non-negative
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_primal_objective(y_test, x_test, w_test, 1)
    49.0
    """
    hinge=hinge_loss(y,X,w)
    primal=np.sum(hinge)+(lambda_/2)*np.sum(w**2)
    return primal

In [214]:
test(calculate_primal_objective)

✅ Your `calculate_primal_objective` passed 4 tests.


In [215]:
def calculate_accuracy(y, X, w):
    """compute the accuracy on the given dataset (X, y) using the model w.
    
    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
    
    Returns:
        scalar, between 0 and 1
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_accuracy(y_test, x_test, w_test)
    0.5
    """
    ypred=(X@w>0)*2-1
    accuracy=np.mean(y==ypred)
    return accuracy

In [216]:
test(calculate_accuracy)

✅ Your `calculate_accuracy` passed 4 tests.


In [109]:
X_b=-1
print((X_b>0)*2)

0


## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [110]:
x1=np.array([[1,2,3],[1,2,3]])
x1.shape
x1.shape[1]

3

In [217]:
def is_support(y_n, x_n, w):
        """a datapoint is support if max{} is not 0. """
        return y_n * x_n @ w <= 1 #include also essential support vectors???

In [218]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    
    Args:
        X: the dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
        lambda_: positive scalar number
        n: the index of the (one) datapoint we have sampled
        num_examples: N
    
    Returns:
        numpy array, shape = (num_features)
    
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_stochastic_gradient(y_test, x_test, w_test, 1, 1, 2)
    array([ 8, 11, 17])
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    grad = - y_n * x_n.T if is_support(y_n, x_n, w) else np.zeros_like(x_n.T)
    grad = num_examples * np.squeeze(grad) + lambda_ * w
    return grad

In [219]:
test(calculate_stochastic_gradient)

✅ Your `calculate_stochastic_gradient` passed 4 tests.


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [220]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma / (it+1) * grad   
        
        if it % 20000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
            
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))


sgd_for_svm_demo(y, X)

iteration=0, cost=61804254.81972499
iteration=20000, cost=243.24688363205237
iteration=40000, cost=213.9963125478125
iteration=60000, cost=143.21021691457227
iteration=80000, cost=132.83394424203541
iteration=100000, cost=130.0373288548718
iteration=120000, cost=127.43412648250006
iteration=140000, cost=240.34956373450282
iteration=160000, cost=153.22896158119178
iteration=180000, cost=124.41970507838889
training accuracy = 0.9261862917398945


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [221]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    
    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        lambda_: positive scalar number
        alpha: vector of dual coordinates, shape = (num_examples)
        w: vector of primal parameters, shape = (num_features)
        n: the coordinate to be updated
        
    Returns:
        w: updated vector of primal parameters, shape = (num_features)
        alpha: updated vector of dual parameters, shape = (num_examples)
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_coordinate_update(y_test, x_test, 1, alpha_test, w_test, 0)
    (array([-0.1,  0.1,  0.3]), array([0.3, 0.1]))
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    alpha[n]=min(max((old_alpha_n+lambda_*(1-y_n*(w.T.dot(x_n)))/(x_n.T.dot(x_n))),0.0),1.0)
    w = w+1.0/lambda_*(alpha[n]-old_alpha_n)*x_n*y_n #why do we remove alpha_old???
    return w, alpha

In [222]:
test(calculate_coordinate_update)

✅ Your `calculate_coordinate_update` passed 5 tests.


In [223]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem.
    
    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        w: vector of primal parameters, shape = (num_features)
        alpha: vector of dual coordinates, shape = (num_examples)
        lambda_: non negative scalar number

    Output:
        scalar

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_dual_objective(y_test, x_test, w_test, alpha_test, 1)
    0.065
    """
    A=np.ones(X.shape[0])
    dual=alpha.T.dot(A)-lambda_/2.0*np.sum(w**2) ## OR np.sum(alpha)
    return dual

In [208]:
alpha_test=np.array([.1,.1])
x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
print(alpha_test.shape)
print(alpha_test.T.dot(np.ones(x_test.shape[0])))

(2,)
0.2


In [224]:
test(calculate_dual_objective)

✅ Your `calculate_dual_objective` passed 5 tests.


In [225]:
# Notice that the gap is going to 0
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 20000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:808.53669, dual:0.01230, gap:808.52439
iteration=20000, primal:142.05236, dual:56.29013, gap:85.76223
iteration=40000, primal:129.43343, dual:88.77455, gap:40.65888
iteration=60000, primal:168.92861, dual:103.53625, gap:65.39236
iteration=80000, primal:124.81478, dual:111.02917, gap:13.78560
iteration=100000, primal:123.36356, dual:115.15537, gap:8.20819
iteration=120000, primal:124.31730, dual:117.76771, gap:6.54959
iteration=140000, primal:144.08663, dual:119.26229, gap:24.82434
iteration=160000, primal:122.85513, dual:120.25221, gap:2.60291
iteration=180000, primal:126.40467, dual:120.87198, gap:5.53269
training accuracy = 0.9279437609841827


We can see that the gap is decreasing towards 0, and compared to SGD implementation, the primal objective is decreasing faster ???